# Czyszczenie danych i przygotowanie zbioru do analizy

W celu wstępnego zorientowania się w formacie otrzymanego pliku, możemy skorzystać z polecenia `head` które wczyta wskazaną ilość wierszy z początku pliku.

In [1]:
file_name = 'TitanicMess.tsv'

!head -n 5 {file_name}

PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked	ship
1	0	3	"Braund, Mr. Owen Harris"	male	22	1	0	"A/5 21171"	7,25		S	Titanic
2	1	1	"Cumings, Mrs. John Bradley (Florence Briggs Thayer)"	female	38	1	0	"PC 17599"	71,2833	C85	C	Titanic
3	1	3	"Heikkinen, Miss. Laina"	female	26	0	0	"STON/O2. 3101282"	7,925		S	Titanic
4	1	1	"Futrelle, Mrs. Jacques Heath (Lily May Peel)"	female	35	1	0	113803	53,1	C123	S	Titanic


## Wczytanie pliku

Na podstawie powyższego wyniku możemy stwierdzić że: 
- kolumny naszego zbioru danych są odzielone tabulatorami
- w zbiorze występują wartości numeryczne, lecz jako separator wykorzystano znak `,`
- zbiór zawiera unikalny identyfikator pasażera

In [2]:
import pandas as pd
import numpy as np
import re
from functools import reduce, partial, partialmethod

In [3]:
read_csv_args = {
    'sep': '\t',
    'index_col': 'PassengerId',
    'decimal': ','
}

df = pd.read_csv(file_name, **read_csv_args)
print('Number of rows: {}, number of columns {}'.format(df.shape[0], df.shape[1]))
df.head()

Number of rows: 892, number of columns 12


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic


## Konwersja typów

Przetwarzanie danych zaczniemy od sprawdzenie poprawności inferencji typów kolumn. Warto zwrócić uwagę na fakt, że wartości typu `string` będą wyświelone jako `object`.

In [4]:
df.dtypes

Survived     int64
Pclass       int64
Name        object
Sex         object
Age         object
SibSp        int64
Parch        int64
Ticket      object
Fare        object
Cabin       object
Embarked    object
ship        object
dtype: object

### Rzutowanie na wartości numeryczne

Możemy stwierdzić, że wartości atrybutów `Age` i `Fare` nie zostały wczytane poprawnie. Mogło to być spowodowane brakującymi wartościami lub niepoprawnym formatem liczb.

W pierwszej komórce zadeklarowano funkcje pomocniczne, a w następnej wykorzystano je do rzutowania liczb. Ostatecznie ponownie sprawdzono typy kolumn.

In [5]:
def compose2(f, g):
    return lambda a: f(g(a))

def compose(*fs):
    return reduce(compose2, reversed(fs))

def apply_regexes(regex_dict):
    def apply_regexes_to_value(value):
        if type(value) is not str:
            return value
        
        return reduce(
            lambda value, regex_replacement: re.sub(*regex_replacement, value),
            regex_dict.items(),
            value
        )
    
    return apply_regexes_to_value


parse_series_to_numeric = compose(
    apply_regexes({ '[a-z]': '', ',': '.' }),
    pd.to_numeric,
)

In [6]:
series_to_parse = ['Age', 'Fare']

for series_name in series_to_parse:
    df[series_name] = df[series_name].apply(parse_series_to_numeric)

In [7]:
df.dtypes

Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
ship         object
dtype: object

In [8]:
df.shape

(892, 12)

## Obsługa brakujących wartości

Za pomocą poniższego fragmentu kodu, sprawdzono ilość wartości `NaN` w każdej kolumnie.

In [9]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         173
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       685
Embarked      2
ship          0
dtype: int64

### Atrybut `Cabin`

Ponieważ ilość brakujących wartości stanowi ponad 75% zbioru danych, postanowiono opuścić tą kolumnę w dalszej analizie.

In [10]:
columns_to_drop = ['Cabin']

df = df.drop(columns=columns_to_drop)

### Atrybut `Age`

Ilość brakujących wartości jest znacząca, lecz stanowi poniżej 20% całego zbioru. Z tego powodu zdecydowano się uzupełnić brakujące wartości średnią.

In [11]:
def replace_nan_with_avg(df, series):
    for series_name in series:
        non_nan = df[series_name].notnull()
        avg = df[series_name][non_nan].mean()
        
        df.loc[~non_nan, series_name] = avg
    
    return df

In [12]:
columns_to_replace_with_avg = ['Age']

df = replace_nan_with_avg(df, columns_to_replace_with_avg)

### Atrybut `Embarked`

Ponieważ istnieją jedynie dwa wiersze pozbawione tej wartości, możemy je opuścić z dalszej analizy.

In [13]:
df = df.dropna()

### Ponownie sprawdzamy ilość brakujących wartości oraz wymiary zbioru

In [14]:
print('Number of rows: {}, number of columns {}'.format(df.shape[0], df.shape[1]))
df.isnull().sum()

Number of rows: 890, number of columns 11


Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
ship        0
dtype: int64

## Analiza wartości kategorycznych

W celu sprawdzenia jakości pozostałych atrybutów, wyświelamy ich unikalne wartości.

In [15]:
print('''
Unique values in columns:
Survived: {}
Pclass: {}
Sex: {}
Embarked: {}
ship: {}
'''.format(
    df['Survived'].unique(),
    df['Pclass'].unique(),
    df['Sex'].unique(),
    df['Embarked'].unique(),
    df['ship'].unique(),
))


Unique values in columns:
Survived: [0 1]
Pclass: [3 1 2]
Sex: ['male' 'female' 'malef' 'mal' 'fem' 'femmale']
Embarked: ['S' 'C' 'Q' 'So' 'Co' 'Qe']
ship: ['Titanic']



### Atrybut `ship`

Jak można zauważyć, atrybut `ship` przyjmuje tą samą wartość dla wszystkich wierszy, przez co nie będzie zbyt użyteczny w analizie danych. Z tego powodu zdecydowano się na jego pominięcie.

In [16]:
columns_to_drop = ['ship']

df = df.drop(columns=columns_to_drop)

### Atrybut `Sex`

Jak widać w wartościach atrybutu istnieją literówki. W celu poprawienia wartości przyjęto huerystykę mówiącą że wartość atrybutu zawierającą `fe` kwalifikujemy jako płeć żeńską.

In [17]:
is_female = lambda sex_str: 'fe' in sex_str
to_sex_str = lambda is_female: 'female' if is_female else 'male'
normalize_sex = compose(
    is_female,
    to_sex_str,
)

df['Sex'] = df['Sex'].apply(normalize_sex)

In [18]:
print('''
Unique values of 'Sex' attribute: {}
Number of female/male passengers: {} / {}
'''.format(
    df['Sex'].unique(),
    (df['Sex'] == 'female').sum(),
    (df['Sex'] == 'male').sum(),
))


Unique values of 'Sex' attribute: ['male' 'female']
Number of female/male passengers: 312 / 578



### Atrybut `Embarked`

Można założyć, że błędnie wprowadzano wartości atrybutu poprzez podawanie pierwszej lub pierwszej i drugiej litery kodującej port. Postanowiono opuścić wszystkie litery kodów poza pierwszą.

In [19]:
normalize_embarked = lambda location_code: location_code[:1]

df['Embarked'] = df['Embarked'].apply(normalize_embarked)

In [20]:
print('''
Unique values of 'Embarked' attribute: {}
Number of S/C/Q passengers: {} / {} / {}
'''.format(
    df['Embarked'].unique(),
    (df['Embarked'] == 'S').sum(),
    (df['Embarked'] == 'C').sum(),
    (df['Embarked'] == 'Q').sum(),
))


Unique values of 'Embarked' attribute: ['S' 'C' 'Q']
Number of S/C/Q passengers: 645 / 168 / 77



## Analiza poprawności wartości numerycznych

W celu sprawdzenia poprawności wartości numerycznych, wyznaczono ich podstawowe statystyki.

In [21]:
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,890.000000,890.000000,890.000000,890.000000,890.00000,890.000000
mean,0.385393,2.310112,35.897546,0.526966,0.37191,31.966488
std,0.486962,0.835388,148.414757,1.103147,0.76913,49.873745
min,0.000000,1.000000,-12.000000,0.000000,0.00000,-90.000000
25%,0.000000,2.000000,22.000000,0.000000,0.00000,7.895800
50%,0.000000,3.000000,32.000000,0.000000,0.00000,14.454200
75%,1.000000,3.000000,35.929166,1.000000,0.00000,30.646850
max,1.000000,3.000000,4435.000000,8.000000,5.00000,512.329200


Można zauważyć, że istnieją nieprawidłowe wartości atrybutu `Age` oraz `Fare`. Założono, że ujemne wartości są błędem który należy poprawić poprzez branie pod uwagę wartości bezwzględnej.

In [22]:
columns_to_abs = ['Age', 'Fare']

df[columns_to_abs] = df[columns_to_abs].abs()

Następnie zauważono nieprawidłowe wartości wieku, które przekraczają rozsądny zakres wartości.
Ponieważ były tylko dwa wiersze w których wiek pasażera przekraczał 150 lat, zdecydowano się je usunąć.

In [23]:
AGE_LIMIT = 150
rows_to_drop = df[df['Age'] > AGE_LIMIT]

df = df.drop(rows_to_drop.index)

In [24]:
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,888.000000,888.000000,888.000000,888.000000,888.000000,888.000000
mean,0.386261,2.309685,30.736279,0.528153,0.372748,32.217656
std,0.487166,0.835943,13.354861,1.104105,0.769794,49.788584
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,7.917700
50%,0.000000,3.000000,32.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,35.929166,1.000000,0.000000,30.771850
max,1.000000,3.000000,80.000000,8.000000,5.000000,512.329200


## Zapis danych

In [25]:
df.to_csv('TitanicCleaned.csv')